In [77]:
import poliastro as pa

import astropy.units as u
from astropy import time

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit

from poliastro.plotting import OrbitPlotter3D
from poliastro.util import norm

import numpy as np

In [78]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

<ScienceState solar_system_ephemeris: 'jpl'>

In [283]:
np.linalg.norm([1,1,1])

1.7320508075688772

In [290]:
class Env():
    def __init__(self):
        
        self.tof_target = 302*u.day
        self.dt = 10*u.day
        
        self.m0 = 525.2*u.kg
        self.T_max = 0.5*u.N
        self.c = 2500*10*u.m/u.s
        
        self.nond_l = (1*u.AU).decompose()
        self.nond_v = ((Sun.k/self.nond_l)**0.5).decompose()
        self.nond_t = (self.nond_l/self.nond_v).decompose()
        
        self.nond_m = self.m0
        
        self.reset()
        
    
    def reset(self):
        
        #t_l  = time of launch
        t_l = self.t_l = time.Time(2454396.30662312, format='jd') #used in linares paper
        
        tof = self.tof = 0.*u.day #((date_launch - self.t_ref)/self.nond_t).decompose().value #non dimensional flight time elapsed
        
        o_i = self.o_i = Orbit.from_body_ephem(Earth, t_l)
        o_f = self.o_f = Orbit.from_body_ephem(Mars,  t_l)

        r,  v  = self.r,  self.v  = o_i.rv() # state of satellite
        rt, vt = self.rt, self.vt = o_f.rv() # t for target
        
        re = self.re = rt - r # e for error
        ve = self.ve = vt - v
        
        m = self.m = self.m0
        
        state = self.state = [r, v, m, re, ve, tof]
        
        return self.n_d_state(state)
    
    def get_target_rv(self, t=None):
        
        # get target r, v at some specified time.
        # if no time specified, use stored (launch time + tof)
        
        if not t:
            t = self.t_l + self.tof
        
        o_f = Orbit.from_body_ephem(Mars,  t)
        
        return o_f.rv()
        
    
    def n_d_state(self, state=None):
        
        if not state:
            state = self.state
            
        r, v, m, re, ve, tof = state
        
        n_r   = (r  /self.nond_l).decompose().value
        n_v   = (v  /self.nond_v).decompose().value
        n_m   = (m  /self.nond_m).decompose().value
        n_re  = (re /self.nond_l).decompose().value
        n_ve  = (ve /self.nond_v).decompose().value
        n_tof = (tof/self.nond_t).decompose().value
        
        n_state = np.concatenate([n_r, n_v, [n_m], n_re, n_ve, [n_tof]])
        
        return n_state
        
    def d_state(self, n_state):
        
        n_r  = n_state[:3]
        n_v  = n_state[3:6]
        n_m  = n_state[6]
        n_re = n_state[7:10]
        n_ve = n_state[10:13]
        n_tof = n_state[13]
        
        r  = (n_r  * self.nond_l).to(u.AU)
        v  = (n_v  * self.nond_v)
        m  = (n_m  * self.nond_m)
        re = (n_re * self.nond_l).to(u.AU)
        ve = (n_ve * self.nond_v)
        tof = (n_tof*self.nond_t).to(u.day)
        
        state = [r, v, m, re, ve, tof]
        
        return state
        
        
    def step(self, n_T):
        
        done = False
        info = None
        
        dt = self.dt
        
        
        # ----- calc new state
        
        # action consists of thrust perc in each axis.
        
        a = (n_T*self.T_max)/(self.m)
        
        def create_ad(a):
            # take in acceleration
            # return dimensional acceleration
            
            def ad(t0, u_, k_):
                
                return a.to(u.km/u.s**2).value
            
            return ad
        
        ad = create_ad(a)
        
        
        #return Sun.k, self.r, self.v, [dt.value]*dt.unit
        rs, vs = pa.twobody.propagation.cowell(Sun.k, r=self.r, v=self.v, tofs=[dt.value]*dt.unit, ad=ad)
        
        r = self.r = rs[0]
        v = self.v = vs[0]
        m = self.m = self.m - (norm(n_T*self.T_max)/self.c)*dt
        
        self.tof += dt
    
        rt, vt = self.rt, self.vt = self.get_target_rv()
        
        re = self.re = rt - r
        ve = self.ve = vt - v
        
        state = self.state = [r, v, m, re, ve, self.tof]
        
        
        # ----- rewards
        
        c1, c2, c3, c4, c5, c6 = 0.5, -500, -0.05, -500, 0.25, 0
        
        norm_n_re = (norm(re)/self.nond_l).decompose().value
        norm_n_ve = (norm(ve)/self.nond_v).decompose().value
     
        reward = - norm_n_re - norm_n_ve
        
        reward += c1*np.exp(c2*norm_n_re)
        reward += c2*np.exp(c4*norm_n_ve)
        
        # mass reward
        reward -= c5*(1-(m/self.nond_m).decompose().value)
        
        # time reward
        reward -= c6*(abs(self.tof_target - self.tof)).to(u.day).value if norm_n_re < 0.25 else 0
        
        # constraints
        
        if np.linalg.norm(n_T) > 1:
            reward -= 20*(np.linalg.norm(n_T))
        
        
        
        # ----- terminal condtions
        
        if norm(r) < 0.25*self.nond_l or norm(r) > 1.75*self.nond_l:
            reward -= 20
            done = True
            info = {'reason':'norm r bounds'}
        
        # terminate if the tof is close to target tof
        # todo: if the dt style is changed, change the dt def here
        if self.tof > 1.5*self.tof_target:
            done = True
            info = {'reason':'tof bounds'}
            
        if self.m < 0.2*self.m0:
            reward -= 20
            done  = True
            info = {'reason': 'm < 0.2*m0'}
        
        
        return self.n_d_state(state), reward, done, info

In [291]:
env = Env()

In [292]:
env.state

[<Quantity [1.30414460e+08, 6.66938419e+07, 2.89007281e+07] km>,
 <Quantity [-1287195.79845256,  2057589.32506894,   891941.97309473] km / d>,
 <Quantity 525.2 kg>,
 <Quantity [-1.89779947e+07,  1.11549866e+08,  4.98163312e+07] km>,
 <Quantity [-450249.60763693, -965527.79509938, -344120.37683866] km / d>,
 <Quantity 0. d>]

In [293]:
env.d_state(env.n_d_state())

[<Quantity [0.87176682, 0.4458208 , 0.19318943] AU>,
 <Quantity [-14898.09951913,  23814.69126237,  10323.40246637] m / s>,
 <Quantity 525.2 kg>,
 <Quantity [-0.12686006,  0.7456648 ,  0.33300161] AU>,
 <Quantity [ -5211.22231061, -11175.09022106,  -3982.87473193] m / s>,
 <Quantity 0. d>]

In [294]:
[env.r, env.v]

[<Quantity [1.30414460e+08, 6.66938419e+07, 2.89007281e+07] km>,
 <Quantity [-1287195.79845256,  2057589.32506894,   891941.97309473] km / d>]

In [295]:
s = env.reset()
R = 0

tof = 0

while tof < env.tof_target:
    #n_T = np.array([0.1,0,0.])
    n_T = np.random.rand(3)
    s, r, d, info = env.step(n_T)
    
    print(r, np.linalg.norm(n_T))
    
    R += r
    
    if d:
        break
        
print(s[-1]*env.nond_t.to(u.day), R, info)

-1.1628726488084977 0.9964096046052693
-1.0679560562337547 0.8655869920420642
-0.9736797133116051 0.7402134151720583
-0.8891130683619697 0.9191269196081318
-23.91666930786979 1.1547009742971948
-0.7635157966845529 0.9515760288091367
-0.715453489056304 0.3138612121541963
-0.6836613325180937 0.755660048247952
-0.687303158769589 0.8752773782078747
-28.237012336182158 1.3765101580821966
-0.7724257480395386 0.3477033689429113
-26.758334835761108 1.2955058752473199
-27.4492457649571 1.324469269752086
-1.0772422189535877 0.6704696334301561
-1.1875456079706945 0.9779806908471139
-22.18059644201944 1.0411609949417076
-25.048964752036117 1.175476557336513
-1.7591723194814217 0.7649649929895617
-2.0037293793497812 0.7554456986863043
-2.3364565300343765 0.9811854298395021
-28.133109934841414 1.2641420430408818
-3.6265783450458877 0.9261362059479005
-25.373952460896167 0.8740650177532372
229.99999999999997 d -226.80459124718294 {'reason': 'norm r bounds'}


In [255]:
env.d_state(s)

[<Quantity [-0.01736569,  1.38105019,  0.59865337] AU>,
 <Quantity [-6714.37131983, -9026.65145535, -3911.65693919] m / s>,
 <Quantity 207.248 kg>,
 <Quantity [ 0.3221528 , -2.64438458, -1.18653383] AU>,
 <Quantity [31286.49837586, 15882.56922015,  6392.5944145 ] m / s>,
 <Quantity 460. d>]

In [163]:
def ad(t,u,k):
    return ((n_T*env.T_max)/(env.m))

In [167]:
ad(0,0,0).to(u.km/u.s**2).value

array([0., 0., 0.])

In [152]:
pa.twobody.propagation.cowell(*vec, ad=ad)

UnitConversionError: Can only apply 'add' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)